<a href="https://colab.research.google.com/github/himanshudas13/MovieRecommenderSystem/blob/main/Collaborative_Filtering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install recommender-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 1.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [38]:
path='/content/drive/MyDrive/ML-DL/movies/ml-latest-small/ml-latest-small/ratings.csv'

In [39]:
def load_movie_list(path):
  user_ratings= pd.read_csv(path)
  user_ratings = user_ratings.sample(n=100, axis=0)
  user_ratings.columns = user_ratings.columns.str.replace("'", "")
  user_ratings.drop('timestamp', axis=1, inplace=True)
  user_ratings_pivot = user_ratings.pivot(index='movieId', columns='userId', values='rating').fillna('NA')
  user_ratings_pivot = user_ratings_pivot.sample(n=50, axis=1)
  return user_ratings_pivot

df=load_movie_list(path)
df.shape

(97, 50)

In [40]:
# Get all movie IDs from the DataFrame
movie_ids = df.index.tolist()

# Create a dictionary for the new user's ratings
my_ratings = {}

# Generate 10 random movie IDs and assign random ratings (1-5)
for _ in range(10):
    movie_id = random.choice(movie_ids)
    rating = random.randint(1, 5)
    my_ratings[movie_id] = rating

# Add the new user's ratings to the DataFrame
new_user_id = 1  # As specified in the prompt
for movie_id, rating in my_ratings.items():
    df.loc[movie_id, new_user_id] = rating

print(f"New user_id: {new_user_id}\n{df}")

New user_id: 1
userId   307 18  605 132 227  448 556  584 232 292  ... 522 70  555 385 140  \
movieId                                             ...                       
1         NA  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
19        NA  NA  NA  NA  NA   NA  NA  3.0  NA  NA  ...  NA  NA  NA  NA  NA   
34        NA  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
50        NA  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
145      3.0  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
...      ...  ..  ..  ..  ..  ...  ..  ...  ..  ..  ...  ..  ..  ..  ..  ..   
132800    NA  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
133419    NA  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
140725    NA  NA  NA  NA  NA  3.0  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
146244    NA  NA  NA  NA  NA   NA  NA   NA  NA  NA  ...  NA  NA  NA  NA  NA   
166528    NA  NA  NA  NA  NA   NA  NA

In [41]:
def normalize_ratings(df):
  df_ = df.replace('NA', np.nan)
  y_mean = df_.mean(axis=1, skipna=True)
  df_ = df_.subtract(y_mean, axis=1)
  return df_, y_mean

In [70]:
df_norm, y_mean = normalize_ratings(df)

In [43]:
df_norm

,1,18,19,27,34,50,57,62,70,89,...,99114,109569,110130,116823,132046,132800,133419,140725,146244,166528
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
num_movies, num_users = df_norm.shape
num_features = 10

# Set initial parameters (W, X), using random initialization
np.random.seed(1234)  # for consistent results
W = np.random.normal(size=(num_users, num_features))
X = np.random.normal(size=(num_movies, num_features))
b = np.random.normal(size=(1, num_users))

In [45]:
import tensorflow as tf

# Convert X, W, and b to TensorFlow variables
X = tf.Variable(X, dtype=tf.float64)
W = tf.Variable(W, dtype=tf.float64)
b = tf.Variable(b, dtype=tf.float64)

# Ensure df_norm is a TensorFlow tensor
df_norm = tf.convert_to_tensor(df_norm, dtype=tf.float64)


In [46]:
import tensorflow as tf

def cost_func(X, W, b, Y, lambda_):
    """
    Returns the cost for the content-based filtering using TensorFlow.

    Args:
      X (tf.Tensor (num_movies,num_features)): matrix of item features
      W (tf.Tensor (num_users,num_features)) : matrix of user parameters
      b (tf.Tensor (1, num_users))           : vector of user parameters
      Y (tf.Tensor (num_movies,num_users))   : matrix of user ratings of movies
      lambda_ (float): regularization parameter

    Returns:
      J (tf.Tensor) : Cost
    """
    # Create a mask for non-NA values in Y (i.e., where ratings exist)
    mask = ~tf.math.is_nan(Y)

    # Compute the prediction error
    prediction_error = (tf.matmul(X, tf.transpose(W)) + b - Y)

    # Apply the mask to only consider valid ratings (ignore NaN values)
    masked_error = tf.where(mask, prediction_error, tf.zeros_like(prediction_error))

    # Calculate the cost function
    J = 0.5 * tf.reduce_sum(masked_error**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))

    return J


In [47]:
# Evaluate cost function with regularization
J = cost_func(X, W, b, df_norm, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 1849.58


In [48]:
import tensorflow as tf
from tensorflow import keras

# Set up your training variables
iterations = 200
lambda_ = 1
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

# Assuming X, W, b, and df_norm are defined as tf.Variable or tf.Tensor
for iter in range(iterations):
    # Use TensorFlow’s GradientTape to record the operations used to compute the cost
    with tf.GradientTape() as tape:
        # Compute the cost (forward pass included in cost)
        cost_value = cost_func(X, W, b, df_norm, lambda_)

    # Use the gradient tape to automatically retrieve the gradients of the trainable variables with respect to the loss
    grads = tape.gradient(cost_value, [X, W, b])

    # Run one step of gradient descent by updating the value of the variables to minimize the loss.
    optimizer.apply_gradients(zip(grads, [X, W, b]))

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value.numpy():.1f}")


Training loss at iteration 0: 1273.4
Training loss at iteration 20: 54.2
Training loss at iteration 40: 12.9
Training loss at iteration 60: 4.8
Training loss at iteration 80: 3.9
Training loss at iteration 100: 3.7
Training loss at iteration 120: 3.7
Training loss at iteration 140: 3.7
Training loss at iteration 160: 3.7
Training loss at iteration 180: 3.7


In [73]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

# Replace NaN values in y_mean with 0 before restoring the mean
y_mean_filled = np.nan_to_num(y_mean.reshape(-1, 1), nan=0.0)

# Restore the mean to get the predicted ratings
pm = p + y_mean_filled

# Predictions for the new user (52 is the column corresponding to the new user)
my_predictions = pm[:, new_user_id - 1]

# Sort predictions in descending order
ix = np.argsort(my_predictions)[::-1]


my_predictions[ix]

array([7.29108119, 5.54426803, 5.20003479, 5.20001989, 5.20001869,
       5.20001276, 5.20000954, 5.20000352, 5.19999082, 5.1999826 ,
       5.19997648, 5.19993434, 4.70013754, 4.70003141, 4.70001915,
       4.7000146 , 4.70000795, 4.69999132, 4.69995667, 4.20006333,
       4.20006207, 4.20003473, 4.20002955, 4.20002625, 4.20001826,
       4.20001563, 4.2000118 , 4.20000275, 4.19999141, 4.19998226,
       4.19997859, 4.19997663, 4.16936165, 3.70005027, 3.70003299,
       3.70003209, 3.70002526, 3.70002485, 3.69998307, 3.23173231,
       3.20013852, 3.20005459, 3.20003199, 3.20002785, 3.20002006,
       3.19999086, 3.19998639, 3.19997815, 3.19997716, 3.19997205,
       3.19997028, 3.19993801, 3.19993286, 3.05064692, 3.05063726,
       2.70004108, 2.70003608, 2.70000224, 2.20001011, 2.1999714 ,
       2.1999202 , 2.05063976, 2.0506355 , 1.8038332 , 1.55064778,
       1.20005052, 0.80386513, 0.70001505, 0.30387808, 0.20007229,
       0.20004973, 0.20004872, 0.20004028, 0.20003744, 0.20002

In [74]:


# Get movie IDs that the new user has already rated
my_rated = my_ratings.keys()

# Display top 17 movie predictions that the user hasn't rated yet
print("Top movie predictions for the new user:\n")
for i in range(17):
    j = ix[i]
    if movie_ids[j] not in my_rated:  # Check if the movie wasn't rated by the new user
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movie_ids[j]}')

# Compare original and predicted ratings for movies the user has rated
print('\n\nOriginal vs Predicted ratings:\n')
for movie_id in my_rated:
    idx = movie_ids.index(movie_id)
    original_rating = my_ratings[movie_id]
    predicted_rating = my_predictions[idx]
    print(f'Original {original_rating}, Predicted {predicted_rating:0.2f} for {movie_ids[idx]}')


Top movie predictions for the new user:

Predicting rating 5.20 for movie 34
Predicting rating 5.20 for movie 3033
Predicting rating 5.20 for movie 235
Predicting rating 5.20 for movie 1982
Predicting rating 5.20 for movie 1234
Predicting rating 5.20 for movie 1254
Predicting rating 5.20 for movie 1258
Predicting rating 5.20 for movie 50
Predicting rating 5.20 for movie 593
Predicting rating 5.20 for movie 380
Predicting rating 4.70 for movie 1610
Predicting rating 4.70 for movie 4993
Predicting rating 4.70 for movie 132800
Predicting rating 4.70 for movie 590
Predicting rating 4.70 for movie 215


Original vs Predicted ratings:

Original 2, Predicted 3.05 for 3098
Original 1, Predicted 0.30 for 5323
Original 2, Predicted 2.05 for 1
Original 2, Predicted 1.55 for 253
Original 1, Predicted 0.80 for 4874
Original 2, Predicted 2.05 for 30812
Original 2, Predicted 3.05 for 153
Original 1, Predicted 1.80 for 36517
Original 4, Predicted 5.54 for 80572
Original 5, Predicted 7.29 for 8784


In [ ]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

In [ ]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[2700] = 5

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[2609] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');